In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
train = train.rename(columns={'3_labels':'label','comment':'text'})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
test = test.rename(columns={'3_labels':'label','comment':'text'})
test.fillna(0, inplace=True)
print(len(test))



1489


In [3]:
train_change_feautre = train.iloc[:, 4:]
test_change_feautre = test.iloc[:, 4:]

train_code_change = train_change_feautre.values
test_code_change = test_change_feautre.values

In [4]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [5]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [6]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['Corrective','Adaptive','Perfective'], sample_size=8)

In [7]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [8]:
len(train)

304

In [9]:
train['label'].value_counts()

label
Perfective    102
Corrective    102
Adaptive      100
Name: count, dtype: int64

In [10]:
test['label'].value_counts()

label
Corrective    501
Perfective    498
Adaptive      490
Name: count, dtype: int64

In [11]:
len(train_dataset)

328

In [12]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [13]:
train_dataset

Dataset({
    features: ['commit_id', 'project', 'text', 'label', 'ADDING_ATTRIBUTE_MODIFIABILITY', 'ADDING_METHOD_OVERRIDABILITY', 'ADDITIONAL_CLASS', 'ADDITIONAL_FUNCTIONALITY', 'ADDITIONAL_OBJECT_STATE', 'ALTERNATIVE_PART_DELETE', 'ALTERNATIVE_PART_INSERT', 'ATTRIBUTE_RENAMING', 'ATTRIBUTE_TYPE_CHANGE', 'CLASS_RENAMING', 'COMMENT_DELETE', 'COMMENT_INSERT', 'COMMENT_MOVE', 'COMMENT_UPDATE', 'CONDITION_EXPRESSION_CHANGE', 'DECREASING_ACCESSIBILITY_CHANGE', 'DOC_DELETE', 'DOC_INSERT', 'DOC_UPDATE', 'INCREASING_ACCESSIBILITY_CHANGE', 'METHOD_RENAMING', 'PARAMETER_DELETE', 'PARAMETER_INSERT', 'PARAMETER_ORDERING_CHANGE', 'PARAMETER_RENAMING', 'PARAMETER_TYPE_CHANGE', 'PARENT_CLASS_CHANGE', 'PARENT_CLASS_DELETE', 'PARENT_CLASS_INSERT', 'PARENT_INTERFACE_CHANGE', 'PARENT_INTERFACE_DELETE', 'PARENT_INTERFACE_INSERT', 'REMOVED_CLASS', 'REMOVED_FUNCTIONALITY', 'REMOVED_OBJECT_STATE', 'REMOVING_ATTRIBUTE_MODIFIABILITY', 'REMOVING_CLASS_DERIVABILITY', 'REMOVING_METHOD_OVERRIDABILITY', 'RETURN_T

In [14]:
train

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Extract Variable,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder
1173,f6892a2e31f7db9fb37caf38b66826ffb496deca,tapiji,Renames `StringLiteralAuditor` to `I18nBuilder...,Perfective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1273,5fe6054a3835b11a4432e3da770f779183de4291,Mylyn Reviews,Minor cleanup\n\n-ignore warning about interna...,Perfective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,84225b56288b90fe708cf37c909cec8be150faae,intellij-community,SVN: configure branches NPE--,Corrective,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
98,6ceadd85dc54ab7e5a58cf51d50289055f3d6bb9,hbase,HBASE-3387 Pair does not deep check arrays for...,Corrective,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
743,88a7e673b3ed7e07aa5cf31a1163697808a1f763,orientdb,"Changed syntax for ""create class"" command. Now...",Adaptive,0,0,0,0,1,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,439b7750d4750722a03d60465da2e4b62bbcf626,spring-framework,"fixed ""hibernateManagedSession"" mode to actual...",Corrective,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1381,258afc8c809f68bd1b530e115725e0bf511e02ca,Delta Spike,DELTASPIKE-402 make DependentProvider Serializ...,Corrective,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1411,f2757568dac2a6473f054101f80ac2b8f291ce4e,Vala,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...,Adaptive,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1779,b52a5a0b2c1166c28049c3debb78e03c6692dedc,adangel$pmd,"optimizations, export improvements\ngit-svn-id...",Perfective,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [15]:
model_id = "all-roberta-large-v1"

In [16]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found in /all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [17]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_change_feautre,
    test_code_change = test_change_feautre,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [18]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 41.76it/s]
***** Running training *****
  Num examples = 13120
  Num epochs = 1
  Total optimization steps = 820
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [06:11<00:00, 371.85s/it]
***** Running evaluation *****


CPU times: user 3min 58s, sys: 2min 55s, total: 6min 53s
Wall time: 6min 17s


{'accuracy': 0.8085963734049698,
 'precision': 0.8114007751941158,
 'recall': 0.8085963734049698,
 'f1': 0.8086608410309232}